## 0. BEM blade dividing

In [2]:
import numpy as np
import math

#==================.exe Compiler#=======================

# Set the range and distribution of r
num_elements = 10  # Number of blade elements
R = 8.5
r_start = 2.473  # Starting radius (m)
r_end = 8.5      # Ending radius (m)
c_start = 1.545  # Starting chord length (m)
c_end = 0.927    # Ending chord length (m)

# Use linspace to generate evenly spaced r values, num_elements + 1 points, forming num_elements segments
delta_r = (r_end - r_start) / num_elements  # Width of each segment δr
radii = np.linspace(r_start, r_end, num_elements + 1)
chord_lengths = np.linspace(c_start, c_end, num_elements + 1)

theta_root = math.radians(10)
theta_tip = math.radians(0)

def get_twist_angle(r, r_start, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * ((r - r_start) / (R - r_start))

print("r (m)    twist (deg)")
for r in radii:
    twist = get_twist_angle(r, r_start, R, theta_root, theta_tip)
    print(f"{r:6.3f}   {math.degrees(twist):6.2f}")
print("Width of each segment δr:", delta_r)
print("Radius values (radii):", radii)
print("Chord length values (chord):", chord_lengths)


r (m)    twist (deg)
 2.473    10.00
 3.076     9.00
 3.678     8.00
 4.281     7.00
 4.884     6.00
 5.486     5.00
 6.089     4.00
 6.692     3.00
 7.295     2.00
 7.897     1.00
 8.500     0.00
Width of each segment δr: 0.6027
Radius values (radii): [2.473  3.0757 3.6784 4.2811 4.8838 5.4865 6.0892 6.6919 7.2946 7.8973
 8.5   ]
Chord length values (chord): [1.545  1.4832 1.4214 1.3596 1.2978 1.236  1.1742 1.1124 1.0506 0.9888
 0.927 ]


## 1. Define U_infinity to get a and a'

In [3]:
import numpy as np
import math

# Parameter settings
U_infinity = 2  # Freestream velocity (m/s)
omega = 1.335   # Rotational speed (rad/s)
R = 8.5  # Fixed total blade radius (m)
B = 3  # Number of blades
tolerance = 1e-6  # Iteration convergence tolerance
small_value = 1e-6  # Small value to prevent division by zero errors

# Initial CL, CD
CL_initial = np.array([0.00000, 0.11060, 0.22060, 0.32930, 0.43620, 0.54070, 0.64220, 0.75230, 0.87660, 0.99990, 1.07670])
CD_initial = np.array([0.00096, 0.00097, 0.00100, 0.00106, 0.00115, 0.00127, 0.00141, 0.00163, 0.00192, 0.00233, 0.00300])

# Twist angle parameters
theta_root = math.radians(10)  # Twist angle at the root (10°)
theta_tip = math.radians(0)   # Twist angle at the tip (0°)

# Calculate the twist angle at a given radius position
def get_twist_angle(r, r_start, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * ((r - r_start) / (R - r_start))

def prandtl_tip_loss(B, r, R, phi):
    if abs(r - R) < small_value:
        return 0.7  # When r = R, F = 0.7
    f = (B / 2) * (R - r) / (r * math.sin(phi + small_value))  # Prevent division by zero
    exp_f = math.exp(-f)
    # Numerical protection to ensure exp_f is within [-1, 1]
    exp_f_clipped = max(min(exp_f, 1.0), -1.0)
    F = (2 / math.pi) * math.acos(exp_f_clipped)
    print(f"Debug: r={r:.3f}, phi={math.degrees(phi):.3f}°, sin(phi)={math.sin(phi):.6f}, f={f:.6f}, exp(-f)={exp_f:.6f}, F={F:.6f}")  # Debug output
    return F

# Create the function to calculate the force projection matrix T
def calculate_forces_projection(CL, CD, phi):
    # Calculate the T matrix and apply it to [CL, CD]
    T = np.array([[math.cos(phi), math.sin(phi)],
                  [math.sin(phi), -math.cos(phi)]])
    forces = np.array([CL, CD])
    projected_forces = T @ forces  # Matrix multiplication
    Cn, Ct = projected_forces
    return Cn, Ct

# Function to calculate induction factors considering twist and new formula
def calculate_induction_with_twist(a_init, a_prime_init, r, chord, CL, CD, consider_tip_loss):
    a = a_init
    a_prime = a_prime_init

    for _ in range(1000):
        # Calculate the twist angle and correct the relative flow angle
        twist_angle = get_twist_angle(r, r_start, R, theta_root, theta_tip)
        phi = math.atan((1 - a) * U_infinity / ((1 + a_prime) * omega * r + small_value)) - twist_angle

        # Calculate Cn and Ct based on the corrected phi value
        Cn, Ct = calculate_forces_projection(CL, CD, phi)

        # Calculate the Prandtl tip loss factor
        if consider_tip_loss:
            F = prandtl_tip_loss(B, r, R, phi)
        else:
            F = 1  # No tip loss considered

        # Calculate Solidity blade loading coefficient sigma
        sigma = chord / (2 * math.pi * r)

        # Update a and a_prime
        a_new = 1 / (1 + F * (4 * math.sin(phi)**2) / (sigma * Cn + small_value))
        a_prime_new = 1 / (F * (4 * math.sin(phi) * math.cos(phi) / (sigma * Ct + small_value)) - 1)

        # Check convergence conditions
        if abs(a_new - a) < tolerance and abs(a_prime_new - a_prime) < tolerance:
            a, a_prime = a_new, a_prime_new
            break
        # Update the induction factors
        a, a_prime = a_new, a_prime_new

    return a, a_prime, F, Cn, Ct

# Initialize result arrays
a_no_tip = []
a_prime_no_tip = []
a_with_tip = []
a_prime_with_tip = []
F_values = []
Cn_initial = []
Ct_initial = []

# For each radius value, calculate the induction factors and tip loss factor
for i, r in enumerate(radii):
    CL = CL_initial[i]
    CD = CD_initial[i]
    chord = chord_lengths[i]

    # Calculate the induction factors without considering tip loss
    a_nt, a_prime_nt, _, Cn, Ct = calculate_induction_with_twist(0.3, 0.01, r, chord, CL, CD, consider_tip_loss=False)
    a_no_tip.append(a_nt)
    a_prime_no_tip.append(a_prime_nt)

    # Calculate the induction factors with considering tip loss
    a_wt, a_prime_wt, F, Cn, Ct = calculate_induction_with_twist(0.3, 0.01, r, chord, CL, CD, consider_tip_loss=True)
    a_with_tip.append(a_wt)
    a_prime_with_tip.append(a_prime_wt)
    F_values.append(F)
    Cn_initial.append(Cn)
    Ct_initial.append(Ct)

# Output results
print("Radius (r)  | a (No Tip Loss) | a' (No Tip Loss) | a (With Tip Loss) | a' (With Tip Loss) | Tip Loss Factor F |    Cn    |    Ct    |    CL   |   CD  ")
for i, r in enumerate(radii):
    print(f"{r:.3f}     | {a_no_tip[i]:.6f}      | {a_prime_no_tip[i]:.6f}       | "
          f"{a_with_tip[i]:.6f}       | {a_prime_with_tip[i]:.6f}      | {F_values[i]:.4f}    | "
          f"{Cn_initial[i]:.6f} | {Ct_initial[i]:.6f} | {CL_initial[i]:.4f}  | {CD_initial[i]:.4f}")


Debug: r=2.473, phi=12.775°, sin(phi)=0.221130, f=16.531726, exp(-f)=0.000000, F=1.000000
Debug: r=2.473, phi=21.207°, sin(phi)=0.361739, f=10.105823, exp(-f)=0.000041, F=0.999974
Debug: r=2.473, phi=21.207°, sin(phi)=0.361741, f=10.105781, exp(-f)=0.000041, F=0.999974
Debug: r=3.076, phi=9.654°, sin(phi)=0.167695, f=15.774947, exp(-f)=0.000000, F=1.000000
Debug: r=3.076, phi=15.341°, sin(phi)=0.264565, f=9.999014, exp(-f)=0.000045, F=0.999971
Debug: r=3.076, phi=16.277°, sin(phi)=0.280280, f=9.438377, exp(-f)=0.000080, F=0.999949
Debug: r=3.076, phi=16.348°, sin(phi)=0.281470, f=9.398478, exp(-f)=0.000083, F=0.999947
Debug: r=3.076, phi=16.353°, sin(phi)=0.281552, f=9.395733, exp(-f)=0.000083, F=0.999947
Debug: r=3.076, phi=16.353°, sin(phi)=0.281558, f=9.395544, exp(-f)=0.000083, F=0.999947
Debug: r=3.076, phi=16.353°, sin(phi)=0.281558, f=9.395531, exp(-f)=0.000083, F=0.999947
Debug: r=3.678, phi=7.763°, sin(phi)=0.135073, f=14.556332, exp(-f)=0.000000, F=1.000000
Debug: r=3.678, ph

In [ ]:
import math
import numpy as np

# Fixed Parameters
U_infinity = 2      # Incoming flow speed (m/s)
omega = 1.355       # Rotational speed (rad/s)
R = 8.5             # Blade tip radius (m)
B = 3               # Number of blades
tolerance = 1e-6    # Convergence tolerance for iterations
small_value = 1e-6  # Small value to prevent division by zero

theta_root = math.radians(10)  # Root twist angle (10°)
theta_tip = math.radians(0)    # Tip twist angle (0°)

# Data for r = R
r = R
chord = 0.927
CL = 1.07670
CD = 0.00300

def get_twist_angle(r, R, theta_root, theta_tip):
    return theta_root - (theta_root - theta_tip) * (r / R)

def prandtl_tip_loss(B, r, R, phi):
    # When r approaches R, directly return assumed F = 0.6
    if abs(r - R) < small_value:
        return 0.6
    f = (B / 2) * (R - r) / (r * math.sin(phi))
    F = (2 / math.pi) * math.acos(math.exp(-f))
    return F

def calculate_induction_with_twist_debug(a_init, a_prime_init, r, chord, CL, CD, consider_tip_loss):
    a = a_init
    a_prime = a_prime_init
    print(f"[Debug Info] r = {r:.3f} m, chord = {chord:.3f} m, CL = {CL}, CD = {CD}")
    for it in range(1000):
        twist_angle = get_twist_angle(r, R, theta_root, theta_tip)
        # Calculate relative flow angle phi (subtracting twist angle)
        phi = math.atan((1 - a) * U_infinity / ((1 + a_prime) * omega * r + small_value)) - twist_angle

        # If considering tip loss, r == R directly return F = 0.5
        if consider_tip_loss:
            F = prandtl_tip_loss(B, r, R, phi)
        else:
            F = 1

        Cn, Ct = calculate_forces_projection(CL, CD, phi)
        sigma = chord / (2 * math.pi * r)

        # Update a and a′ values
        a_new = 1 / (1 + F * (4 * math.sin(phi)**2) / (sigma * Cn + small_value))
        a_prime_new = 1 / (F * (4 * math.sin(phi) * math.cos(phi) / (sigma * Ct + small_value)) - 1)

        print(f"Iteration {it:2d}: φ = {phi:.6f} rad, a = {a_new:.6f}, a' = {a_prime_new:.6f}, F = {F:.4f}")

        # Convergence check
        if abs(a_new - a) < tolerance and abs(a_prime_new - a_prime) < tolerance:
            a, a_prime = a_new, a_prime_new
            break
        a, a_prime = a_new, a_prime_new

    return a, a_prime, F, Cn, Ct

print("----- Iteration Debug Process with Tip Loss (r=R, F=0.5) -----")
a_with_tip, a_prime_with_tip, F_with_tip, Cn_with_tip, Ct_with_tip = calculate_induction_with_twist_debug(
    0.3, 0.01, r, chord, CL, CD, consider_tip_loss=True
)
print("\nConverged Result (with tip loss): a = {:.6f}, a' = {:.6f}, F = {:.4f}".format(a_with_tip, a_prime_with_tip, F_with_tip))


----- 考虑尖端损失（r=R时 F=0.5）时的迭代调试过程 -----
【调试信息】 r = 8.500 m, chord = 0.927 m, CL = 1.0767, CD = 0.003
迭代  0: φ = 0.119775 rad, a = 0.351366, a' = 0.007724, F = 0.6000
迭代  1: φ = 0.111309 rad, a = 0.385531, a' = 0.007703, F = 0.6000
迭代  2: φ = 0.105493 rad, a = 0.411296, a' = 0.007687, F = 0.6000
迭代  3: φ = 0.101102 rad, a = 0.432064, a' = 0.007674, F = 0.6000
迭代  4: φ = 0.097560 rad, a = 0.449672, a' = 0.007664, F = 0.6000
迭代  5: φ = 0.094554 rad, a = 0.465225, a' = 0.007655, F = 0.6000
迭代  6: φ = 0.091898 rad, a = 0.479445, a' = 0.007646, F = 0.6000
迭代  7: φ = 0.089468 rad, a = 0.492849, a' = 0.007638, F = 0.6000
迭代  8: φ = 0.087177 rad, a = 0.505835, a' = 0.007630, F = 0.6000
迭代  9: φ = 0.084956 rad, a = 0.518743, a' = 0.007622, F = 0.6000
迭代 10: φ = 0.082748 rad, a = 0.531890, a' = 0.007614, F = 0.6000
迭代 11: φ = 0.080498 rad, a = 0.545604, a' = 0.007605, F = 0.6000
迭代 12: φ = 0.078150 rad, a = 0.560251, a' = 0.007595, F = 0.6000
迭代 13: φ = 0.075642 rad, a = 0.576273, a' = 0.007585, F

## 2. Calculate Relative velocity **Urel** and Flow angle **phi**

In [4]:
# Initialize the lists to store intermediate results
one_minus_a_U_infinity = []
one_plus_a_prime_omega_r = []
Urel_values = []
tan_phi_values = []
phi_values = []
phi_minus_twist_values = []
twist_angles_deg = []  # New: list to store twist angles in degrees

# Calculate intermediate variables for each radial position
for i, r in enumerate(radii):
    a = a_no_tip[i]
    a_prime = a_prime_no_tip[i]
    chord = chord_lengths[i]
    CL = CL_initial[i]
    CD = CD_initial[i]

    one_minus_a_Uinf = (1 - a) * U_infinity
    one_plus_a_prime_omega_r_val = (1 + a_prime) * omega * r

    Urel = math.sqrt(one_minus_a_Uinf**2 + one_plus_a_prime_omega_r_val**2)

    tan_phi = one_minus_a_Uinf / (one_plus_a_prime_omega_r_val + small_value)
    phi = math.degrees(math.atan(tan_phi))

    twist_angle = get_twist_angle(r, r_start, R, theta_root, theta_tip)
    twist_angle_deg = math.degrees(twist_angle)
    phi_minus_twist = phi - twist_angle_deg

    one_minus_a_U_infinity.append(one_minus_a_Uinf)
    one_plus_a_prime_omega_r.append(one_plus_a_prime_omega_r_val)
    Urel_values.append(Urel)
    tan_phi_values.append(tan_phi)
    phi_values.append(phi)
    twist_angles_deg.append(twist_angle_deg)
    phi_minus_twist_values.append(phi_minus_twist)

print("\nRadius (r)  | (1-a)U∞  | (1+a')ωr |   U_rel  | tan(phi) |  phi(deg) | twist(deg)| phi - twist(r)")
for i, r in enumerate(radii):
    print(f"{r:.3f}     | {one_minus_a_U_infinity[i]:.6f} | {one_plus_a_prime_omega_r[i]:.6f} | "
          f"{Urel_values[i]:.6f} | {tan_phi_values[i]:.6f} | {phi_values[i]:.6f} | {twist_angles_deg[i]:8.6f} | {phi_minus_twist_values[i]:.6f}")



Radius (r)  | (1-a)U∞  | (1+a')ωr |   U_rel  | tan(phi) |  phi(deg) | twist(deg)| phi - twist(r)
2.473     | 1.999864 | 3.301240 | 3.859746 | 0.605792 | 31.207134 | 10.000000 | 21.207134
3.076     | 1.949783 | 4.114892 | 4.553459 | 0.473836 | 25.353260 | 9.000000 | 16.353260
3.678     | 1.874038 | 4.927469 | 5.271808 | 0.380325 | 20.823037 | 8.000000 | 12.823037
4.281     | 1.764280 | 5.739094 | 6.004155 | 0.307414 | 17.088179 | 7.000000 | 10.088179
4.884     | 1.587726 | 6.549763 | 6.739456 | 0.242410 | 13.626206 | 6.000000 | 7.626206
5.486     | 0.115168 | 7.355373 | 7.356274 | 0.015658 | 0.897045 | 5.000000 | -4.102955
6.089     | 0.359371 | 8.171999 | 8.179897 | 0.043976 | 2.518010 | 4.000000 | -1.481990
6.692     | 0.513173 | 8.980694 | 8.995344 | 0.057142 | 3.270426 | 3.000000 | 0.270426
7.295     | 1.356213 | 9.786689 | 9.880212 | 0.138577 | 7.889649 | 2.000000 | 5.889649
7.897     | 1.514666 | 10.595117 | 10.702837 | 0.142959 | 8.135813 | 1.000000 | 7.135813
8.500     | 1.6216

## 3. Lift & Drag and Normal & Tangential Force

In [7]:
# ====================== Force Calculation Section ======================
rho = 1025  # Water density (kg/m³)
CL_star = np.array([1.1812, 1.3106, 1.8220, 1.7370, 1.5985, 1.4654, 1.3702, 1.3050, 1.2589, 1.1811, 0.7782])
CD_star = np.array([0.37960, 0.16813, 0.03441, 0.01943, 0.01494, 0.01283, 0.01154, 0.01074, 0.01017, 0.00951, 0.00693])

# Initialize storage lists
lift_initial, drag_initial = [], []
Fn_initial, Ft_initial = [], []
lift_star, drag_star = [], []
Fn_star, Ft_star = [], []
Cn_star_values, Ct_star_values = [], []  # Direct use of * versions of Cn and Ct

# Calculate forces for each radius position
for i, r in enumerate(radii):
    Urel = Urel_values[i]  # Ensure Urel_values have been calculated via BEM
    chord = chord_lengths[i]

    # Calculate Cn* and Ct*
    phi = math.atan((1 - a_with_tip[i]) * U_infinity / ((1 + a_prime_with_tip[i]) * omega * r))
    Cn_star, Ct_star = calculate_forces_projection(CL_star[i], CD_star[i], phi)
    Cn_star_values.append(Cn_star)
    Ct_star_values.append(Ct_star)

    # **Initial Force Calculation (using Cn_initial and Ct_initial)**
    L_initial = 0.5 * rho * Urel**2 * chord * delta_r * CL_initial[i] / 1000
    D_initial = 0.5 * rho * Urel**2 * chord * delta_r * CD_initial[i] / 1000
    Fn_initial_val = Cn_initial[i] * 0.5 * rho * Urel**2 * chord * delta_r / 1000
    Ft_initial_val = Ct_initial[i] * 0.5 * rho * Urel**2 * chord * delta_r / 1000

    lift_initial.append(L_initial)
    drag_initial.append(D_initial)
    Fn_initial.append(Fn_initial_val)
    Ft_initial.append(Ft_initial_val)

    # **Secondary Force Calculation (using CL_star/CD_star)**
    L_star = 0.5 * rho * Urel**2 * chord * delta_r * CL_star[i] / 1000
    D_star = 0.5 * rho * Urel**2 * chord * delta_r * CD_star[i] / 1000

    Fn_star_val = Cn_star * 0.5 * rho * Urel**2 * chord * delta_r / 1000
    Ft_star_val = Ct_star * 0.5 * rho * Urel**2 * chord * delta_r / 1000

    lift_star.append(L_star)
    drag_star.append(D_star)
    Fn_star.append(Fn_star_val)
    Ft_star.append(Ft_star_val)

# ====================== Output Calculation Results ======================
print("Radius(r) |     L (kN)     |     D (kN)    |    Fn (kN)   |     Ft (kN)   |    Cn    |    Ct    |    CL    |  CD  |")
for i, r in enumerate(radii):
    print(f"{r:.3f}  | {lift_initial[i]:.6f} kN   | {drag_initial[i]:.6f} kN   | "
          f"{Fn_initial[i]:.6f} kN   | {Ft_initial[i]:.6f} kN   | "
          f"{Cn_initial[i]:.6f} | {Ct_initial[i]:.6f} | {CL_initial[i]:.4f}  | {CD_initial[i]:.4f}")

print("Radius(r) |     L* (kN)    |     D* (kN)    |    Fn* (kN)   |    Ft* (kN)   |    Cn*   |    Ct*   |    CL*  | CD*  |")
for i, r in enumerate(radii):
    print(f"{r:.3f}  | {lift_star[i]:.6f} kN   | {drag_star[i]:.6f} kN   | "
          f"{Fn_star[i]:.6f} kN   | {Ft_star[i]:.6f} kN   | "
          f"{Cn_star_values[i]:.6f} | {Ct_star_values[i]:.6f} | {CL_star[i]:.4f}  | {CD_star[i]:.4f}")


Radius(r) |     L (kN)     |     D (kN)    |    Fn (kN)   |     Ft (kN)   |    Cn    |    Ct    |    CL    |  CD  |
2.473  | 0.000000 kN   | 0.006825 kN   | 0.002469 kN   | -0.006363 kN   | 0.000347 | -0.000895 | 0.0000  | 0.0010
3.076  | 1.050589 kN   | 0.009214 kN   | 1.010681 kN   | 0.286961 kN   | 0.106399 | 0.030210 | 0.1106  | 0.0010
3.678  | 2.691759 kN   | 0.012202 kN   | 2.627336 kN   | 0.585505 kN   | 0.215320 | 0.047984 | 0.2206  | 0.0010
4.281  | 4.985421 kN   | 0.016048 kN   | 4.911160 kN   | 0.857427 kN   | 0.324395 | 0.056635 | 0.3293  | 0.0011
4.884  | 7.942152 kN   | 0.020939 kN   | 7.874707 kN   | 1.033052 kN   | 0.432496 | 0.056737 | 0.4362  | 0.0011
5.486  | 11.170837 kN   | 0.026238 kN   | 11.126042 kN   | -0.999741 kN   | 0.538532 | -0.048390 | 0.5407  | 0.0013
6.089  | 15.584869 kN   | 0.034218 kN   | 15.544518 kN   | -1.121280 kN   | 0.640537 | -0.046204 | 0.6422  | 0.0014
6.692  | 20.916194 kN   | 0.045319 kN   | 20.885157 kN   | -1.139926 kN   | 0.751184 | -0.

<ipython-input-7-262eedb6cf10>:19: RuntimeWarning: invalid value encountered in scalar divide
  phi = math.atan((1 - a_with_tip[i]) * U_infinity / ((1 + a_prime_with_tip[i]) * omega * r))


In [6]:
# Calculate the average values of adjacent rows to obtain 10 rows of results
avg_radii = []
avg_lift_star = []
avg_drag_star = []
avg_Fn_star = []
avg_Ft_star = []
avg_Cn_star = []
avg_Ct_star = []

for i in range(len(radii) - 1):  # Prevent out-of-bound index
    avg_radii.append((radii[i] + radii[i + 1]) / 2)
    avg_lift_star.append((lift_star[i] + lift_star[i + 1]) / 2)
    avg_drag_star.append((drag_star[i] + drag_star[i + 1]) / 2)
    avg_Fn_star.append((Fn_star[i] + Fn_star[i + 1]) / 2)
    avg_Ft_star.append((Ft_star[i] + Ft_star[i + 1]) / 2)
    avg_Cn_star.append((Cn_star_values[i] + Cn_star_values[i + 1]) / 2)
    avg_Ct_star.append((Ct_star_values[i] + Ct_star_values[i + 1]) / 2)

# Output the 10-row average results
print("\nAverage 10-row results:")
print("Radius (r) | Average Lift δL* (N) | Average Drag δD* (N) | Average Normal Force δFn* (N) | Average Tangential Force δFt* (N)")
for i in range(len(avg_radii)):  # Avoid out-of-bound index
    print(f"{avg_radii[i]:.3f}     | {avg_lift_star[i]:.6f} N     | {avg_drag_star[i]:.6f} N     | "
          f"{avg_Fn_star[i]:.6f} N      | {avg_Ft_star[i]:.6f} N")


Average 10-row results:
Radius (r) | Average Lift δL* (N) | Average Drag δD* (N) | Average Normal Force δFn* (N) | Average Tangential Force δFt* (N)
2.774     | 10.423580 N     | 2.147922 N     | 10.257563 N      | 2.965229 N
3.377     | 17.340702 N     | 1.008468 N     | 16.431672 N      | 5.699078 N
3.980     | 24.264623 N     | 0.357015 N     | 23.075977 N      | 7.478250 N
4.582     | 27.701030 N     | 0.283090 N     | 26.786335 N      | 7.018772 N
5.185     | 29.689966 N     | 0.268544 N     | nan N      | nan N
5.788     | 31.763512 N     | 0.272560 N     | nan N      | nan N
6.391     | 34.767418 N     | 0.289328 N     | nan N      | nan N
6.993     | 38.081505 N     | 0.310387 N     | nan N      | nan N
7.596     | 40.601367 N     | 0.327446 N     | nan N      | nan N
8.199     | 35.434203 N     | 0.297916 N     | 35.247605 N      | 3.639661 N


## 4. Output csv file



In [ ]:
import pandas as pd
from google.colab import files

output_file = 'Fast_BEM_Force_Solver_results.csv'

Fast_BEM_results = pd.DataFrame({
    "Element number": list(range(0, num_elements + 1)),
    "radii": radii,
    "a": a_no_tip,
    "a'": a_prime_no_tip,
    "a_corr": a_with_tip,
    "a'_corr": a_prime_with_tip,
    "F_tiploss": F_values,
    "(1-a)U_infnty": one_minus_a_U_infinity,
    "(1+a')wr": one_plus_a_prime_omega_r,
    "Urel": Urel_values,
    "tan(phi)": tan_phi_values,
    "phi": phi_values,
    "phi-twist": phi_minus_twist_values,
    "L*": lift_star,
    "D*": drag_star,
    "L": lift_initial,
    "D": drag_initial,
    "Fn*": Fn_star,
    "Ft*": Ft_star,
    "Fn": Fn_initial,
    "Ft": Ft_initial,
    "Cn*": Cn_star_values,
    "Ct*": Ct_star_values,
    "Cn": Cn_initial,
    "Ct": Ct_initial,
    "CL*": CL_star,
    "CD*": CD_star,
    "CL_ini": CL_initial,
    "CD_ini": CD_initial,
})

Fast_BEM_results.to_csv(output_file, index=False)

try:
    from google.colab import files
    files.download(output_file)
except:
    print(f"File saved: {output_file}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>